In [1]:
# importing the required libraries
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date, datetime, timedelta
import re

In [2]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./json/pass.json', scope)

# authorize the clientsheet
client = gspread.authorize(creds)

In [5]:
# get the instance of the Spreadsheet
sheet = client.open('Samsung - Geral Dia')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(1)

In [6]:
today_minus = f'{date.today()-timedelta(days=1)}|{date.today()-timedelta(days=2)}|{date.today()-timedelta(days=3)}' 
current_date_1 =  f'{date.today()-timedelta(days=1)}'

In [7]:
pattern = re.compile(r'(%s)'%today_minus)
cell_list = sheet_instance.findall(pattern)

In [8]:
mask = f'R{cell_list[0].row}C{cell_list[0].col}:R{cell_list[-1].row}C9'

In [18]:
google_ads = sheet_instance.get(mask)

In [19]:
df = pd.DataFrame(google_ads)

In [23]:
dictr = {0: 'date',
         3: 'campaign',
         7: 'cost',
         6: 'impressions'}
 
# call rename () method
df.rename(columns=dictr,
          inplace=True)

In [24]:
filters = ['cad1-','cad3-','cad12-','cad26-','cad42-']
final = pd.DataFrame()
for i in filters:
    df3 = df.where(df['campaign'].str.contains(i)).dropna()
    df3['midia'] = 'facebook'
    group = df3.rename(columns = {'cost':f'cost_{i}', 'impressions':f'impressions_{i}'})
    final = final.append(group.groupby(['date','midia'])[[f'cost_{i}',f'impressions_{i}']].apply(lambda x : x.astype(float).sum()).reset_index())

/tmp/ipykernel_1423/1201763459.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(group.groupby(['date','midia'])[[f'cost_{i}',f'impressions_{i}']].apply(lambda x : x.astype(float).sum()).reset_index())
/tmp/ipykernel_1423/1201763459.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(group.groupby(['date','midia'])[[f'cost_{i}',f'impressions_{i}']].apply(lambda x : x.astype(float).sum()).reset_index())
/tmp/ipykernel_1423/1201763459.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(group.groupby(['date','midia'])[[f'cost_{i}',f'impressions_{i}']].apply(lambda x : x.astype(float).sum()).reset_index())
/tmp/ipykernel_1423/1201763459.py:7: FutureWarning: The fr

In [25]:
final

,date,midia,cost_cad1-,impressions_cad1-,cost_cad3-,impressions_cad3-,cost_cad12-,impressions_cad12-,cost_cad26-,impressions_cad26-,cost_cad42-,impressions_cad42-
0,2022-04-24,facebook,2637.6,168420.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-04-24,facebook,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
0,2022-04-24,facebook,NaN,NaN,NaN,NaN,NaN,NaN,3497.66,358987.0,NaN,NaN
